In [ ]:
import os

import numpy as np

from tensorflow.keras.models import load_model

from autoencoder import Autoencoder
from helper_funcs import *

import pandas

import optuna
from optuna.visualization import plot_pareto_front, plot_optimization_history, plot_slice

from params import lsh_test, hypercube_test, kmeans_test, gnn_test, mrng_test, nsg_test, get_aaf

In [ ]:
models = os.listdir('./models/')

dataset = 'MNIST/input.dat'
query = 'MNIST/query.dat'

model_to_files = {}
for i, model in enumerate(models):
    normalized_dataset = 'MNIST/' + models[i].removesuffix('.keras') + '_normalized_dataset.dat'
    normalized_query   = 'MNIST/' + models[i].removesuffix('.keras') + '_normalized_query.dat'
    encoded_dataset    = 'MNIST/' + models[i].removesuffix('.keras') + '_encoded_dataset.dat'
    encoded_query      = 'MNIST/' + models[i].removesuffix('.keras') + '_encoded_query.dat'
    decoded_dataset    = 'MNIST/' + models[i].removesuffix('.keras') + '_decoded_dataset.dat'
    decoded_query      = 'MNIST/' + models[i].removesuffix('.keras') + '_decoded_query.dat'

    model_to_files.update({models[i] : [normalized_dataset, normalized_query,
                                        encoded_dataset, encoded_query,
                                        decoded_dataset, decoded_query]})

n = 60000

In [ ]:
for model in model_to_files:
    normalized_dataset, normalized_query, encoded_dataset, encoded_query, decoded_dataset, decoded_query = model_to_files[model]

    model = 'models/' + model

    # load model
    autoencoder = load_model(model)
    shape = autoencoder.layers[-2].output_shape[1:] # get shape of encoded layer

    # load dataset
    x_train = load_dataset(dataset)
    x_train = x_train.astype('float32') / 255.
    x_test = load_dataset(query)
    x_test = x_test.astype('float32') / 255.
    if len(shape) == 3: # if model type is convolutional
        x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
        x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))
    else:
        x_train = np.reshape(x_train, (len(x_train), 784))
        x_test = np.reshape(x_test, (len(x_test), 784))

    encoded_train = autoencoder.encode(x_train)
    encoded_test = autoencoder.encode(x_test)

    # deflatten encoded datasets
    encoded_train = deflatten_encoded(encoded_train, shape)
    encoded_test = deflatten_encoded(encoded_test, shape)

    # decode encoded datasets
    decoded_train = autoencoder.decode(encoded_train)
    decoded_test = autoencoder.decode(encoded_test)

    # save original datasets normalized
    save_decoded_binary(x_train, normalized_dataset)
    save_decoded_binary(x_test, normalized_query)

    # normalize encoded datasets
    encoded_train = normalize(encoded_train)
    encoded_test = normalize(encoded_test)

    # save encoded datasets
    save_encoded_binary(encoded_train, encoded_dataset)
    save_encoded_binary(encoded_test, encoded_query)

    # normalize decoded datasets
    decoded_train = normalize(decoded_train)
    decoded_test = normalize(decoded_test)

    # save decoded datasets
    save_decoded_binary(decoded_train, decoded_dataset)
    save_decoded_binary(decoded_test, decoded_query)

In [ ]:
def objective_lsh(trial):
    param_dict = {'model': trial.suggest_categorical('model', model_to_files.keys()),
                  'k': trial.suggest_int('k', 1, 10),
                  'L': trial.suggest_int('L', 1, 10),
                  'table_size':  trial.suggest_categorical('table_size', [int(n/32), int(n/16), int(n/8)]),
                  'window_size': trial.suggest_int('window_size', 0.01, 1),
                  'query_trick': trial.suggest_categorical('query_trick', [True, False])
                 }
    
    print("Trial parameters:", param_dict)

    encoded_dataset, encoded_query = model_to_files[param_dict['model']][2:4]

    average_time, aaf_latent, min_neighbors = lsh_test(encoded_dataset, encoded_query, queries_num=100, **param_dict, N=60)

    # trial should return at least 60 neighbors to be used in GNNS
    # penalize model if slower than brute force
    c0 = - min_neighbors.value + 60
    c1 = average_time.value - 1e-4
    trial.set_user_attr('constraint', (c0, c1))

    return aaf_latent.value, average_time.value

def constraints(trial):
    return trial.user_attrs['constraint']

In [ ]:
%%time
sampler = optuna.samplers.NSGAIISampler(constraints_func=constraints)
lsh_study = optuna.create_study(study_name='lsh', directions=['minimize', 'minimize'], sampler=sampler)
lsh_study.optimize(objective_lsh, n_trials=100, n_jobs=-1)
print("-------------------- Best trials --------------------")
trials = sorted(lsh_study.best_trials, key=lambda x: x.values)
# print feasible trials only
for trial in trials:
    print("Trial no. {}".format(trial.number))
    print(" Values = {}, Constraints = {}".format(trial.values, trial.user_attrs["constraint"]))
    print(" Params = {}".format(trial.params))

In [ ]:
df = lsh_study.trials_dataframe()

df_sorted = df.copy(deep=True)
df_sorted = df_sorted.dropna(subset=['value'])
df_sorted = df_sorted.sort_values(by=['value'], ascending=True)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted

In [ ]:
plot_pareto_front(lsh_study, target_names=['aaf', 'average_time'])

In [ ]:
plot_optimization_history(lsh_study, target = lambda t: t.values[0], target_name = 'aaf')

In [ ]:
plot_optimization_history(lsh_study, target = lambda t: t.values[1], target_name = 'average_time')

In [ ]:
plot_slice(lsh_study, target = lambda t: t.values[0], target_name = 'aaf')

In [ ]:
plot_slice(lsh_study, target = lambda t: t.values[1], target_name = 'average_time')